In [1]:
import glob
import io
import os
import string

In [4]:
f = open('./data/IMDb_train.tsv', 'w')
path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        text = text.replace('\t', " ")
        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)
        
path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

In [5]:
# テストデータの作成

f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [ ]:
import re
import string

In [6]:
print("区切り文字：", string.punctuation)

区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [7]:
import string
import re

# 以下の記号はスペースに置き換えます（カンマ、ピリオドを除く）。
# punctuationとは日本語で句点という意味です
print("区切り文字：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 前処理


def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）


def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認します
print(tokenizer_with_preprocessing('I like cats.'))


区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


In [8]:
import torchtext

In [9]:
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [10]:
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))

訓練および検証のデータ数 25000
1つ目の訓練および検証のデータ {'Text': ['sandra', 'bernhard', 'is', 'quite', 'a', 'character', ',', 'and', 'certainly', 'one', 'of', 'the', 'funniest', 'women', 'on', 'earth', '.', 'she', 'began', 'as', 'a', 'stand', 'up', 'comedienne', 'in', 'the', '1970s', ',', 'but', 'her', 'big', 'break', 'came', 'in', '1983', 'when', 'she', 'starred', 'opposite', 'jerry', 'lewis', 'and', 'robert', 'de', 'niro', 'in', 'scorsese', 's', 'underrated', 'masterpiece', ',', 'the', 'king', 'of', 'comedy', '.', 'her', 'film', 'career', 'never', 'quite', 'took', 'off', ',', 'though', '.', 'she', 'did', 'make', 'a', 'couple', 'of', 'odd', 'but', 'entertaining', 'pictures', ',', 'such', 'as', 'dallas', 'doll', '1994', 'or', 'dinner', 'rush', '2000', ',', 'but', 'the', 'most', 'amazing', 'parts', 'were', 'those', 'she', 'created', 'for', 'herself', '.', 'without', 'you', 'i', 'm', 'nothing', 'is', 'undoubtedly', 'her', 'best', 'effort', '.', 'it', 's', 'an', 'adaptation', 'of', 'her', 'smash', 'hit', 'off

In [11]:
import random
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))


訓練データの数 20000
検証データの数 5000
1つ目の訓練データ {'Text': ['shaggy', 'scooby', 'doo', 'get', 'a', 'clue', '.', 'it', 's', 'like', 'watching', 'a', 'much', 'loved', 'relative', 'in', 'the', 'final', 'throws', 'of', 'a', 'degenerative', 'illness', 'clearly', 'the', 'work', 'of', 'people', 'with', 'no', 'soul', ',', 'no', 'love', 'or', 'respect', 'for', 'the', 'original', 'work', '.', 'what', 'in', 'the', 'name', 'of', 'all', 'that', 's', 'holy', 'were', 'they', 'thinking', 'of', 'it', 'seems', 'they', 'were', 'trying', 'to', 'go', 'all', 'post', 'modern', 'and', 'ironic', '.', 'instead', 'it', 's', 'just', 'abysmal', 'swill', 'what', 's', 'the', 'point', 'in', 'taking', 'up', 'a', 'successful', 'franchise', 'like', 'scooby', 'doo', 'if', 'you', 'just', 'going', 'to', 'flush', 'it', 'down', 'the', 'toilet', 'my', 'son', 'loves', 'the', 'original', 'series', 'and', 'even', 'some', 'of', 'the', 'spin', 'off', 'movies', 'but', 'he', 'can', 't', 'stand', 'this', 'drivel', 'and', 'let', 's', 'face', 'it',

In [12]:
# torchtextで単語ベクトルとして英語学習済みモデルを読み込みます

from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')


# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", english_fasttext_vectors.dim)
print("単語数：", len(english_fasttext_vectors.itos))


100%|█████████▉| 999150/999994 [01:06<00:00, 15329.06it/s]

1単語を表現する次元数： 300
単語数： 999994


In [13]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 17916個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


torch.Size([17921, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f0eb0ac6590>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             '.': 5,
             ',': 6,
             'and': 7,
             'a': 8,
             'of': 9,
             'to': 10,
             'is': 11,
             'it': 12,
             'in': 13,
             'i': 14,
             'this': 15,
             'that': 16,
             's': 17,
             'was': 18,
             'as': 19,
             'for': 20,
             'movie': 21,
             'with': 22,
             'but': 23,
             'film': 24,
             't': 25,
             'you': 26,
             'on': 27,
             'not': 28,
             'he': 29,
             'are': 30,
             'his': 31,
             'have': 32,
             'be': 33,
             'one': 34,
             'all': 35,
             'at': 36,
             'they': 37,

In [14]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[    2, 13787,     0,  ...,     1,     1,     1],
        [    2,    14,   143,  ...,     1,     1,     1],
        [    2,    15,    11,  ...,     1,     1,     1],
        ...,
        [    2,    14,   214,  ...,     1,     1,     1],
        [    2,    15,    21,  ...,     1,     1,     1],
        [    2,  1618,     0,  ...,     1,     1,     1]]), tensor([154, 231,  97, 216, 107, 256, 101, 189, 256,  56, 256, 144, 229,  66,
        238, 256, 108, 198, 204,  87, 256, 245, 201, 140]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


In [16]:
batch.Text[0].size()

torch.Size([24, 256])

In [17]:
batch.Text[1].size()

torch.Size([24])